# 1. Setup project

### Load and Visualize AOI from a Local Shapefile (GeoPandas + Folium + Earth Engine)

This section of the notebook performs the following steps:

1. **Check for the presence of shapefiles** in the local directory (`../data/aoi/`).
2. **Authenticate and initialize Google Earth Engine (GEE)** to enable access to remote sensing datasets.
3. **Dynamically import a helper script** (`aoi_loader.py`) containing utility functions to load shapefiles as:

   * a `GeoDataFrame` (for local use and visualization),
   * and an `ee.Geometry` object (for Earth Engine processing).
4. **Load the Area of Interest (AOI)** from the shapefile as both:

   * a `GeoDataFrame` (`aoi_gdf`) using GeoPandas, and
   * an Earth Engine geometry (`aoi_ee`) for cloud-based geospatial analysis.
5. **Create an interactive map** using `folium`, centered on the AOI, and add the AOI layer to it.
6. **Display the map directly within the notebook** (and optionally save it as an HTML file for external viewing).

> 💡 This setup allows smooth interoperability between local GIS data and cloud processing via Earth Engine.

---

> ** Note: Earth Engine Authentication**
>
> When you run `ee.Authenticate()` for the first time, a browser window opens asking you to log in to your Google account and authorize Earth Engine access.
> Once authorized, a **credentials token is saved locally** so that you don’t have to log in again every time.
>
> **Where is the token stored?**
>
> * **Windows:**
>   `C:\Users\<YourUsername>\.config\earthengine\credentials`
>
> * **Linux/macOS:**
>   `~/.config/earthengine/credentials`
>
> On subsequent runs, Earth Engine loads the saved credentials automatically.
>
> **To force re-authentication**, you can:
>
> * Delete or rename the `credentials` file manually, or
> * Use:
>
>   ```python
>   ee.Authenticate(clear_credentials=True)
>   ```

In [24]:
import os
import sys
import importlib.util

import ee
import folium
import geemap.foliumap as geemap

import geopandas as gpd

from IPython.display import display

In [25]:

print(os.path.exists("../data/aoi/aoi_1km7x16_3035.shp"))

ee.Authenticate()
ee.Initialize()

True


In [26]:
def load_module(name, rel_path):
    module_path = os.path.abspath(rel_path)
    spec = importlib.util.spec_from_file_location(name, module_path)
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    spec.loader.exec_module(module)
    return module

sys.path.append(os.path.abspath("../scripts"))

aoi_loader      = load_module("aoi_loader", "../scripts/aoi_loader.py")
download_s1     = load_module("download_s1", "../scripts/download_s1.py")
conv_db_sm     = load_module("conv_db_sm", "../scripts/conv_db_sm.py")




In [27]:
print(os.path.exists("../data/aoi/aoi_1km7x16_3035.shp"))

aoi_gdf = aoi_loader.load_shapefile_as_gdf("../data/aoi/aoi_1km7x16_3035.shp")
print(type(aoi_gdf))

center = aoi_gdf.to_crs(epsg=4326).geometry.centroid.iloc[0]
print(center)
center_coords = [center.y, center.x]

m = folium.Map(location=center_coords, zoom_start=12)
folium.GeoJson(aoi_gdf).add_to(m)

m.save("map_aoi.html")
print("💾 Saved 'map_aoi.html' in the current directory")

from IPython.display import display
display(m)


True
<class 'geopandas.geodataframe.GeoDataFrame'>
POINT (12.201048660903087 44.458029809288234)
💾 Saved 'map_aoi.html' in the current directory


C:\Users\Minutella Francesco\AppData\Local\Temp\ipykernel_22500\2775533220.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = aoi_gdf.to_crs(epsg=4326).geometry.centroid.iloc[0]


In [28]:
# Earth Engine AOI
aoi_ee = aoi_loader.load_shapefile_as_ee("../data/aoi/aoi_1km7x16_3035.shp")

img = download_s1.get_s1_vv_image(aoi_ee, "2024-04-01", "2024-09-30")

Map = geemap.Map(center=[center_coords[0], center_coords[1]], zoom=12)
Map.addLayer(img, {"min": -25, "max": 0}, "SAR VV (Apr–Sep)")
Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")
Map


In [29]:
img_clipped = img.clip(aoi_ee)

Map = geemap.Map(center=center_coords, zoom=12)
Map.addLayer(img_clipped, {"min": -25, "max": 0}, "SAR VV Clipped")
Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")
Map


In [30]:
task = ee.batch.Export.image.toDrive(
    image=img_clipped,
    description='export_sar_vv_clipped',
    folder='earthengine',  # 📁 Google Drive folder
    fileNamePrefix='sarvv2024_04_09',
    scale=10,  # 10m
    region=aoi_ee,
    fileFormat='GeoTIFF'
)

task.start()
print("🚀 Export in GoogleDrive complite: https://code.earthengine.google.com/tasks ")


🚀 Export in GoogleDrive complite: https://code.earthengine.google.com/tasks 


In [31]:
a = -4.33
b = -38.3

soil_moisture_img = conv_db_sm.vv_to_soil_moisture(img_clipped, a, b)


In [32]:
Map = geemap.Map(center=center_coords, zoom=12)
Map.addLayer(soil_moisture_img, {"min": 0, "max": 50, "palette": ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']}, "Soil Moisture (%)")
#Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")

legend_dict = {
    '0–10%':  '#ffffcc',
    '10–20%': '#a1dab4',
    '20–30%': '#41b6c4',
    '30–40%': '#2c7fb8',
    '40–50%': '#253494'
}
Map.add_legend(title="Umidità del suolo (%)", legend_dict=legend_dict)
print(legend_dict)

Map

{'0–10%': '#ffffcc', '10–20%': '#a1dab4', '20–30%': '#41b6c4', '30–40%': '#2c7fb8', '40–50%': '#253494'}


In [ ]:

sand_0_5 = ee.Image("projects/soilgrids-isric/sand_mean").select("sand_0-5cm_mean").divide(10)
silt_0_5 = ee.Image("projects/soilgrids-isric/silt_mean").select("silt_0-5cm_mean").divide(10)
clay_0_5 = ee.Image("projects/soilgrids-isric/clay_mean").select("clay_0-5cm_mean").divide(10)

sand_clipped = sand_0_5.clip(aoi_ee)
silt_clipped = silt_0_5.clip(aoi_ee)
clay_clipped = clay_0_5.clip(aoi_ee)


soil_layers = [
    ("sand_0_5", sand_clipped),
    ("silt_0_5", silt_clipped),
    ("clay_0_5", clay_clipped)
]

for name, image in soil_layers:
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder='earthengine',
        region=aoi_ee,
        scale=250,
        maxPixels=1e13
    )
    task.start()
    print(f"✅ Esportazione avviata per: {name}")


✅ Esportazione avviata per: sand_0_5
✅ Esportazione avviata per: silt_0_5
✅ Esportazione avviata per: clay_0_5


In [ ]:

Map = geemap.Map(center=center_coords, zoom=12)

sand_palette = ['#ffffcc','#c2e699','#78c679','#31a354','#006837']
silt_palette = ['#f7fcb9','#addd8e','#31a354']
clay_palette = ["#ecc052","#c68120","#A34C00","#6b2608"]

Map.addLayer(sand_clipped, {"min": 0, "max": 100, "palette": sand_palette}, "Sand 0–5cm")
Map.addLayer(silt_clipped, {"min": 0, "max": 100, "palette": silt_palette}, "Silt 0–5cm")
Map.addLayer(clay_clipped, {"min": 0, "max": 100, "palette": clay_palette}, "Clay 0–5cm")

Map.addLayer(ee.FeatureCollection(aoi_ee), {}, "AOI")

Map


In [ ]:
rpoints = aoi_loader.load_shapefile_as_gdf("../data/aoi/random_points_3035.shp")

center = rpoints.to_crs(epsg=4326).geometry.centroid.iloc[0]
center_coords = [center.y, center.x]

m = folium.Map(location=center_coords, zoom_start=12)

for _, row in rpoints.iterrows():
    geom = row.geometry
    if geom.geom_type == 'Point':
        lon, lat = geom.x, geom.y
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            color='orange',
            fill=True,
            fill_color='orange',
            fill_opacity=0.8
        ).add_to(m)

rpoints_ee = geemap.gdf_to_ee(rpoints)

aoi_gdf_ee = geemap.gdf_to_ee(aoi_gdf)

Map = geemap.Map(center=center_coords, zoom=12)

Map = geemap.Map(center=center_coords, zoom=12)

Map.addLayer(soil_moisture_img, {"min": 0, "max": 50, "palette": ["#d9f7fe","#a3b4ff","#4650c1","#031563"]}, "Soil Moisture")
Map.addLayer(img_clipped, {"min": -25, "max": 0}, "SAR VV")
Map.addLayer(ee.FeatureCollection(aoi_gdf_ee), {}, "AOI")
Map.addLayer(rpoints_ee, {"color": "orange"}, "Random Points")

Map


C:\Users\Minutella Francesco\AppData\Local\Temp\ipykernel_22500\2541796719.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = rpoints.to_crs(epsg=4326).geometry.centroid.iloc[0]


In [ ]:

soil_stack = sand_clipped.rename("sand").addBands(
    silt_clipped.rename("silt")).addBands(
    clay_clipped.rename("clay"))

sampled = soil_stack.sampleRegions(
    collection=rpoints_ee,
    scale=250,
    geometries=True
)

task = ee.batch.Export.table.toDrive(
    collection=sampled,
    description='rpoints_soil',
    folder='earthengine',
    fileFormat='SHP'
)
task.start()

print("🚀 Export avviato! Controlla: https://code.earthengine.google.com/tasks")


🚀 Export avviato! Controlla: https://code.earthengine.google.com/tasks


In [ ]:

soil_stack = sand_clipped.rename("sand").addBands(
    silt_clipped.rename("silt")).addBands(
    clay_clipped.rename("clay"))

sampled = soil_stack.sampleRegions(
    collection=rpoints_ee,
    scale=250,
    geometries=True
)


rpoints_soil = aoi_loader.load_shapefile_as_gdf("../output/isric/rpoints_soil.shp")


def texture_class_usda(sand, clay, silt):
    if sand is None or silt is None or clay is None:
        return 'n'
    if sand >= 0 and sand <= 45 and clay >= 40 and clay <= 100 and silt >= 0 and silt <= 40:
        return 'Cl'
    elif sand <= 65 and sand >= 45 and clay >= 35 and clay <= 55 and silt >= 0 and silt <= 20:
        return 'SaCl'
    elif sand >= 0 and sand <= 20 and clay >= 40 and clay <= 60 and silt >= 40 and silt <= 60:
        return 'SiCl'
    elif sand >= 20 and sand <= 45 and clay >= 25 and clay <= 40 and silt >= 15 and silt <= 55:
        return 'ClLo'
    elif sand >= 0 and sand <= 20 and clay >= 25 and clay <= 40 and silt >= 40 and silt <= 75:
        return 'SiClLo'
    elif sand >= 45 and sand <= 80 and clay >= 20 and clay <= 35 and silt >= 0 and silt <= 25:
        return 'SaClLo'
    elif sand >= 25 and sand <= 55 and clay >= 5 and clay <= 25 and silt >= 25 and silt <= 50:
        return 'Lo'
    elif sand >= 85 and sand <= 100 and clay >= 0 and clay <= 10 and silt >= 0 and silt <= 15:
        return 'Sa'
    elif sand >= 70 and sand <= 90 and clay >= 0 and clay <= 15 and silt >= 0 and silt <= 30:
        return 'LoSa'
    elif sand >= 45 and sand <= 85 and clay >= 0 and clay <= 20 and silt >= 0 and silt <= 50:
        return 'SaLo'
    elif sand >= 0 and sand <= 50 and clay >= 0 and clay <= 25 and silt >= 50 and silt <= 85:
        return 'SiLo'
    elif sand >= 0 and sand <= 20 and clay >= 0 and clay <= 15 and silt >= 80 and silt <= 100:
        return 'y'
    else:
        return 'n'


rpoints_soil['texture_class'] = rpoints_soil.apply(
    lambda row: texture_class_usda(row['sand'], row['clay'], row['silt']), axis=1
)


rpoints_soil.head()


,sand,SL_ACT,SL_POT,silt,rand_point,cell_id,clay,usda,geometry,texture_class
0,11.0,0,0,48.9,0,4496_2374,40.0,SiCl,POINT (12.19604 44.46095),SiCl
1,11.1,0,0,49.7,1,4496_2374,39.1,SiClLo,POINT (12.20775 44.45635),SiClLo
2,11.0,0,0,49.9,2,4496_2374,39.1,SiClLo,POINT (12.20437 44.45406),SiClLo
3,8.6,0,0,47.8,3,4496_2374,43.6,SiCl,POINT (12.19942 44.46324),SiCl
4,10.9,0,0,49.5,4,4496_2374,39.6,SiClLo,POINT (12.20129 44.45406),SiClLo
